# MTG Price Predictor

## About/Goals: 

The idea of this project is to create an ML model that can take a card's data and return the value of the card should be, based on previous cards it has analysed. This requires NLP processing for the text box, and utilizes tensorflow to build the model.

Cards are evaluated purely based on what a person looking at it for the first time can see - year, text, color, etc.. nothing about format legalities, special flags, or anythingn else of the sort. Also, price is based on standard, nonfoil variant only.

# Data Importation and Cleaning

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [5]:
def save(file):
    file.to_pickle('data/updated_data.pkl')

def load():
    df = pd.read_pickle('data/updated_data.pkl')
    return df # band-aid work-around

In [8]:
df = pd.read_json("data/default_cards_08_05_2025.json")

In [9]:
df.columns

Index(['object', 'id', 'oracle_id', 'multiverse_ids', 'mtgo_id', 'arena_id',
       'tcgplayer_id', 'cardmarket_id', 'name', 'lang', 'released_at', 'uri',
       'scryfall_uri', 'layout', 'highres_image', 'image_status', 'image_uris',
       'mana_cost', 'cmc', 'type_line', 'oracle_text', 'colors',
       'color_identity', 'keywords', 'produced_mana', 'legalities', 'games',
       'reserved', 'game_changer', 'foil', 'nonfoil', 'finishes', 'oversized',
       'promo', 'reprint', 'variation', 'set_id', 'set', 'set_name',
       'set_type', 'set_uri', 'set_search_uri', 'scryfall_set_uri',
       'rulings_uri', 'prints_search_uri', 'collector_number', 'digital',
       'rarity', 'card_back_id', 'artist', 'artist_ids', 'illustration_id',
       'border_color', 'frame', 'full_art', 'textless', 'booster',
       'story_spotlight', 'prices', 'related_uris', 'purchase_uris',
       'mtgo_foil_id', 'power', 'toughness', 'flavor_text', 'edhrec_rank',
       'penny_rank', 'all_parts', 'promo_types

can drop so many of these columns, only need ones that are useful.

### Column Cleaning

In [10]:
len(df)

108955

In [11]:
df.value_counts("variation")
df.value_counts("oversized")

oversized
False    108229
True        726
Name: count, dtype: int64

In [12]:
df = df[(df["variation"]==False) & (df["reprint"]==False) & (df["oversized"]==False) & (df["promo"]==False) & (df["full_art"]==False) & (df["textless"] == False) & (df["content_warning"]!=True)]

In [13]:
len(df) # Dropped about 40k entries

41387

In [14]:
doubles = df["card_faces"].dropna()
doubles.iloc[0] ## this is gonna be tough to work with

[{'object': 'card_face',
  'name': "Obyra's Attendants",
  'mana_cost': '{4}{U}',
  'type_line': 'Creature — Faerie Wizard',
  'oracle_text': 'Flying',
  'power': '3',
  'toughness': '4',
  'flavor_text': "Obyra's devoted servants shrieked as their sleeping mistress slashed at them, unseeing.",
  'artist': 'Andreas Zafiratos',
  'artist_id': 'e2f13a9a-57c5-40de-81d4-3b0723899cdf',
  'illustration_id': 'd1ea5321-62e2-4894-a79f-03b792daf2c8'},
 {'object': 'card_face',
  'name': 'Desperate Parry',
  'mana_cost': '{1}{U}',
  'type_line': 'Instant — Adventure',
  'oracle_text': 'Target creature gets -4/-0 until end of turn. (Then exile this card. You may cast the creature later from exile.)',
  'artist': 'Andreas Zafiratos',
  'artist_id': 'e2f13a9a-57c5-40de-81d4-3b0723899cdf'}]

For first iteration of model, will be removing the multi-faced cards

In [15]:
df = df[df["card_faces"].isna()]

In [16]:
def is_legal(x):
    if 'legal' in x.values():
        return True
    else:
        return False

In [17]:
df["playable"] = df["legalities"].apply(is_legal)

In [18]:
df = df[df["playable"] == True] # remove unnplayable cards

In [19]:
unneeded = ['object', 'id', 'oracle_id', 'multiverse_ids', 'mtgo_id', 'arena_id',
       'tcgplayer_id', 'cardmarket_id', #'name',
         'lang', 'uri',
       'scryfall_uri', 'layout', 'highres_image', 'image_status', 'image_uris', 'legalities', 'games',
       'reserved', 'game_changer', 'finishes', 'oversized',
       'promo', 'reprint', 'variation', 'set_id', 'set', 'set_name',
       'set_type', 'set_uri', 'set_search_uri', 'scryfall_set_uri',
       'rulings_uri', 'prints_search_uri', 'collector_number', 'digital', 'card_back_id', 'artist', 'artist_ids', 'illustration_id',
       'border_color', 'frame', 'full_art', 'textless', 'booster',
       'story_spotlight', 'related_uris', 'purchase_uris',
       'mtgo_foil_id', 'flavor_text', 'edhrec_rank',
       'penny_rank', 'all_parts', 'promo_types', 'security_stamp', 'preview', 'watermark', 'frame_effects', 'loyalty',
       'printed_name', 'tcgplayer_etched_id', 'flavor_name',
       'attraction_lights', 'color_indicator', 'printed_type_line',
       'printed_text', 'variation_of', 'life_modifier', 'hand_modifier',
       'content_warning', 'defense', 'card_faces', 'foil', 'nonfoil'
       , 'playable', 'color_identity']
df = df.drop(unneeded, axis=1)

now that I've cleaned out cards and columns that aren't needed, I need to figure out the best way to transform this data into something that the ML model can actually use. 

For instance, the "type_line" column will have to be split up into various super and subtypes, probably using categorical encoding.

### Type Labeling + Encoding

In [20]:
df["type_line"].describe() # has 4246 unique types currently

count       35098
unique       3120
top       Instant
freq         3592
Name: type_line, dtype: object

In [21]:
"""
df["creature_type"] = df["type_line"].apply(lambda x: x[10:] if "Creature" in x else "NaN")
df["planeswalker_type"] = df["type_line"].apply(lambda x: x[24:] if "Planeswalker" in x else "NaN")
df["kindred_type"] = df["type_line"].apply(lambda x: x.split()[-1] if "Kindred" in x or "Tribal" in x else "NaN")"""
# Found a better way!

'\ndf["creature_type"] = df["type_line"].apply(lambda x: x[10:] if "Creature" in x else "NaN")\ndf["planeswalker_type"] = df["type_line"].apply(lambda x: x[24:] if "Planeswalker" in x else "NaN")\ndf["kindred_type"] = df["type_line"].apply(lambda x: x.split()[-1] if "Kindred" in x or "Tribal" in x else "NaN")'

In [22]:
def filter_subtype(x):
    if "—" in x:
        ind = x.index("—")
        types = x[ind+1:].split()
        return types
    else:
        return []

In [23]:
def filter_maintype(x):
    types = ["Artifact", "Land", "Battle", "Creature", "Enchantment", "Planeswalker", "Instant", "Sorcery"]
    cur = []
    for type in types:
        if type in x:
            cur.append(type)
    return cur

In [24]:
df = df[~df['type_line'].str.contains('Basic')] # remove basic lands

In [25]:
df["legendary"] = df["type_line"].apply(lambda x: 1 if "Legendary" in x else 0)
df["subtype"] = df["type_line"].apply(filter_subtype)
df["main_type"]=df["type_line"].apply(filter_maintype)

In [26]:
from sklearn.preprocessing import LabelEncoder

In [27]:
df["price"] = df["prices"].str["usd"].astype(float) 
df = df.drop("prices", axis=1)

In [28]:
df = df.dropna(subset=["price"])
df = df[df["main_type"].map(len)>0]  # had to filter out bad cards with other types such as conspiracies and stickers

In [29]:
le = LabelEncoder()
le.fit(["Artifact", "Land", "Battle", "Creature", "Enchantment", "Planeswalker", "Instant", "Sorcery"])

LabelEncoder()

In [30]:
df["main_type"] = df["main_type"].apply(lambda x: le.transform(x)) 

In [31]:
le2 = LabelEncoder()

In [32]:
df["subtype"].values

array([list(['Sliver']), list(['Kor', 'Soldier']),
       list(['Siren', 'Pirate']), ..., list([]),
       list(['Faerie', 'Rogue']), list(['Vampire', 'Soldier'])],
      dtype=object)

In [33]:
subtypes = []
for unique in df["subtype"].values:
    if unique != []:
        for val in unique:
            if val not in subtypes:
                subtypes.append(val)

le2.fit(subtypes)


LabelEncoder()

In [34]:
df["subtype"] = df["subtype"].apply(lambda x: le2.transform(x)) # slow, probably better way to do this 

In [35]:
df = df.drop("type_line", axis=1)

### Date -> Year

In [36]:
df["year"] = df["released_at"].apply(lambda x: x.year)
df = df.drop("released_at", axis=1)

### Mana Cost Breakdown

- Number of Pips
- Is X spell?

In [37]:
df["is_x"] = df["mana_cost"].apply(lambda x: 1 if r"{X}" in x else 0)
#df.loc[df["is_x"] == 1]

In [38]:
#print("{1}{W/R}{G}{G}".replace("{", "").replace("}", " ").split())  ->  df

def pip_counter(x):
    new = x.replace("{", "").replace("}", " ").split()
    count = 0
    for x in new:
        if x.isdigit() == False and x != "X":
            count += 1

    return count

In [39]:
df["pip_count"] = df["mana_cost"].apply(pip_counter)
df = df.drop("mana_cost", axis=1)
#df.sort_values(by="pip_count", ascending=False)

### Oracle Text Breakdown -> NLP ? Or can try a parsing method to turn text into columns 

- activated ability?
- etb effect?

In [40]:
df["oracle_text"] = df["oracle_text"].apply(lambda x: x.lower().replace("\n", ". "))

#### Main Phrases

In [41]:
from sklearn.feature_extraction.text import CountVectorizer

using vectorizer to figure out most common substrings

In [42]:
vectorizer = CountVectorizer(ngram_range=(4, 7), lowercase=True, stop_words=None)

In [43]:
X = vectorizer.fit_transform(df["oracle_text"])

In [44]:
sum_words = X.sum(axis=0)
word_freq = [(word, sum_words[0, idx]) for word, idx in vectorizer.vocabulary_.items()]
word_freq = sorted(word_freq, key=lambda x: x[1], reverse=True)

In [45]:
common_phrases = pd.DataFrame(word_freq, columns=['phrase', 'count'])
common_phrases.iloc[:10]

,phrase,count
0,until end of turn,5990
1,at the beginning of,3261
2,when this creature enters,3049
3,gets until end of,2131
4,gets until end of turn,2130
5,the beginning of your,1869
6,at the beginning of your,1814
7,card from your graveyard,1593
8,creature gets until end,1570
9,creature gets until end of,1570


In [46]:
PHRASES = { # starter phrases
    r"when.*enters": "etb",
    r"until end of turn": "eot",
    r"beginning of .* upkeep": "b_o_u",
    r"search .* library": "tutor",
    r"without paying": "free",
    r"whenever .* attacks": "a_t",
    r"deals combat damage": "c_d_t",
    r"look at the tzo.*p": "s_s_t",
    r"return.*graveyard.*battlefield": "reanimate",
    r"when.*(this|a).*dies": "o_d_t",
    r"when.*(this|a).*leaves.": "l_b_t",
}

In [47]:
import re

In [48]:
def canonicalize_text(str):
    for phr, rep in PHRASES.items():
        if re.search(phr, str) != None:
            str = re.sub(phr, rep, str)
    return str

In [49]:
df["oracle_text"] = df["oracle_text"].apply(canonicalize_text)

In [50]:
import tensorflow as tf

In [51]:
from tensorflow.keras.layers import TextVectorization

In [52]:
text_vectorizer = TextVectorization(
    max_tokens=1000,
    output_mode='int',
    ngrams = (2,6)
)

In [53]:
text_vectorizer.adapt(df["oracle_text"])

### Rarity Encoding

In [54]:
rare_encoder = LabelEncoder()

In [55]:
df["rarity"].value_counts()

rarity
rare        11445
common       9878
uncommon     8967
mythic       1987
special         2
Name: count, dtype: int64

In [56]:
df["rarity"] = df["rarity"].apply(lambda x: "common" if x == "special" else x) # the two special cards have common rarity on scryfall

In [57]:
df["rarity"] = rare_encoder.fit_transform(df["rarity"])

### Keyword Encoding

In [58]:
keywrd_encoder = LabelEncoder()

In [59]:
keywords = [] # taking code from earlier
for unique in df["keywords"].values:
    if unique != []:
        for val in unique:
            if val not in keywords:
                keywords.append(val)

keywrd_encoder.fit(keywords)


LabelEncoder()

In [60]:
df["keywords"] = df["keywords"].apply(lambda x: keywrd_encoder.transform(x))

### Color Identity Encoding (one more time!)

definitely could've done a for loop for each column i wanted to encode but oh well it's a little late for that

In [61]:
df

,name,cmc,oracle_text,colors,keywords,produced_mana,rarity,power,toughness,legendary,subtype,main_type,price,year,is_x,pip_count
1,Fury Sliver,6.0,all sliver creatures have double strike.,[R],[],NaN,3,3,3,0,[310],[2],0.30,2006,0,1
2,Kor Outfitter,2.0,"etb, you may attach target equipment you contr...",[W],[],NaN,0,2,2,0,"[183, 315]",[2],0.14,2009,0,2
4,Siren Lookout,3.0,"flying. etb, it explores. (reveal the top card...",[U],"[230, 198]",NaN,0,1,2,0,"[305, 259]",[2],0.08,2017,0,1
7,Surge of Brilliance,2.0,paradox — draw a card for each spell you've ca...,[U],"[390, 238]",NaN,3,NaN,NaN,0,[],[4],0.20,2023,0,1
9,Venerable Knight,1.0,"o_d_t, put a +1/+1 counter on target knight yo...",[W],[],NaN,3,2,1,0,"[157, 181]",[2],0.14,2019,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108944,Morkrut Banshee,5.0,"morbid — etb, if a creature died this turn, ta...",[B],[366],NaN,3,4,4,0,[325],[2],0.09,2011,0,2
108947,Deeproot Historian,4.0,merfolk and druid cards in your graveyard have...,[G],[],NaN,2,3,3,0,"[204, 97]",[2],0.15,2023,0,1
108949,Aggressive Biomancy,2.0,create x tokens that are copies of target crea...,"[G, U]",[215],NaN,2,NaN,NaN,0,[],[7],0.19,2024,1,2
108952,Faerie Bladecrafter,3.0,"flying. o_d_t, each opponent loses x life and ...",[B],[230],NaN,2,2,2,0,"[114, 280]",[2],1.29,2023,0,1


In [62]:
c_i = LabelEncoder()

In [63]:
c_i.fit(["W", "G", "R", "B", "U"])

LabelEncoder()

In [64]:
c_i.transform(["W", "U"])

array([4, 3])

In [65]:
df["colors"] = df["colors"].apply(lambda x: c_i.transform(x))

### Produced Mana 

changing this to binary value if it does(n't)

In [66]:
df["produced_mana"] = df["produced_mana"].replace(pd.NA, 0)

In [67]:
df["produced_mana"] = df["produced_mana"].apply(lambda x: 1 if x != 0 else x)

### Final Step: Replace Power/Toughness NaN with -1

In [68]:
df["power"] = pd.to_numeric(df["power"], errors="coerce").fillna(-1)
df["toughness"] = pd.to_numeric(df["toughness"], errors="coerce").fillna(-1)

In [69]:
df.sort_values(by="price", ascending=False)[:100]

,name,cmc,oracle_text,colors,keywords,produced_mana,rarity,power,toughness,legendary,subtype,main_type,price,year,is_x,pip_count
42945,The Tabernacle at Pendrell Vale,0.0,"all creatures have ""at the b_o_u, destroy this...",[],[],0,2,-1.0,-1.0,1,[],[5],2700.00,1994,0,0
8431,Mishra's Workshop,0.0,{t}: add {c}{c}{c}. spend this mana only to ca...,[],[],1,2,-1.0,-1.0,0,[],[5],2414.49,1994,0,0
108622,Bazaar of Baghdad,0.0,"{t}: draw two cards, then discard three cards.",[],[],0,3,-1.0,-1.0,0,[],[5],2132.00,1993,0,0
37635,Goblin King,3.0,other goblins get +1/+1 and have mountainwalk.,[2],[],0,2,2.0,2.0,0,[137],[2],1499.99,1993,0,2
41737,Raging River,2.0,whenever one or more creatures you control att...,[2],[],0,2,-1.0,-1.0,0,[],[3],1450.00,1993,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39145,Instill Energy,1.0,enchant creature. enchanted creature can attac...,[1],[174],0,3,-1.0,-1.0,0,[21],[3],149.99,1993,0,1
65131,Ice Storm,3.0,destroy target land.,[1],[],0,3,-1.0,-1.0,0,[],[7],147.97,1993,0,1
92984,Fire Elemental,5.0,,[2],[],0,3,5.0,4.0,0,[104],[2],147.33,1993,0,2
89955,Pyramids,6.0,{2}: choose one —. • destroy target aura attac...,[],[],0,2,-1.0,-1.0,0,[],[0],146.66,1993,0,0


In [70]:
test = df.drop(["name", "oracle_text", "colors", "keywords", "subtype", "main_type"], axis=1)

In [71]:
test.head()

,cmc,produced_mana,rarity,power,toughness,legendary,price,year,is_x,pip_count
1,6.0,0,3,3.0,3.0,0,0.30,2006,0,1
2,2.0,0,0,2.0,2.0,0,0.14,2009,0,2
4,3.0,0,0,1.0,2.0,0,0.08,2017,0,1
7,2.0,0,3,-1.0,-1.0,0,0.20,2023,0,1
9,1.0,0,3,2.0,1.0,0,0.14,2019,0,1


In [72]:
test.dtypes

cmc              float64
produced_mana      int64
rarity             int64
power            float64
toughness        float64
legendary          int64
price            float64
year               int64
is_x               int64
pip_count          int64
dtype: object

In [73]:
corr = test.corr()

In [74]:
corr["price"] # shows how much i need the categorical data, there's barely any correlation!!!

cmc             -0.024103
produced_mana    0.031057
rarity           0.028865
power           -0.024330
toughness       -0.022597
legendary        0.017238
price            1.000000
year            -0.114793
is_x            -0.002419
pip_count       -0.016000
Name: price, dtype: float64

## Model Building